In [2]:
import joblib
import os
import pandas as pd
import numpy as np

In [3]:
NOMBRE_CARPETA = 'Student Performance'

try:
    # Construye las rutas completas a los archivos
    ruta_scaler = os.path.join(NOMBRE_CARPETA, 'scaler.joblib')
    ruta_poly_features = os.path.join(NOMBRE_CARPETA, 'poly_features.joblib')
    ruta_modelo_lineal = os.path.join(NOMBRE_CARPETA, 'modelo_regresion_lineal.joblib')
    ruta_modelo_polinomico = os.path.join(NOMBRE_CARPETA, 'modelo_regresion_polinomico.joblib')

    # Carga los objetos desde los archivos
    scaler = joblib.load(ruta_scaler)
    poly_features = joblib.load(ruta_poly_features)
    linear_model = joblib.load(ruta_modelo_lineal)
    poly_model = joblib.load(ruta_modelo_polinomico)

    print(f"\n✅ Modelos y preprocesadores cargados exitosamente desde la carpeta '{NOMBRE_CARPETA}'.")

except FileNotFoundError:
    print(f"\n❌ ERROR: No se encontraron los archivos en la carpeta '{NOMBRE_CARPETA}'.")
    print("   Asegúrate de que este cuaderno esté en el mismo directorio que la carpeta 'Student Performance'.")


✅ Modelos y preprocesadores cargados exitosamente desde la carpeta 'Student Performance'.


In [4]:
def predecir_calificacion_final(G1, G2, studytime, failures, absences):
    """
    Realiza una predicción de la calificación final (G3) utilizando los modelos cargados.
    
    Args:
        G1 (int): Calificación del primer periodo (0-20).
        G2 (int): Calificación del segundo periodo (0-20).
        studytime (int): Tiempo de estudio semanal (1: <2h, 2: 2-5h, 3: 5-10h, 4: >10h).
        failures (int): Número de clases reprobadas anteriormente (0-4).
        absences (int): Número de ausencias escolares (0-93).
        
    Returns:
        dict: Un diccionario con las predicciones de ambos modelos.
    """
    try:
        # Crear un DataFrame con los datos de entrada
        # El orden de las columnas DEBE ser el mismo que en el entrenamiento
        columnas_ordenadas = ['G1', 'G2', 'studytime', 'failures', 'absences']
        datos_nuevos = pd.DataFrame([[G1, G2, studytime, failures, absences]], columns=columnas_ordenadas)
        
        print("\n--------------------------------------------------")
        print("Datos de entrada del estudiante:")
        print(datos_nuevos)
        
        # --- Preprocesamiento ---
        # 1. Escalar los datos con el scaler cargado
        datos_escalados = scaler.transform(datos_nuevos)
        
        # --- Predicción con Modelo Lineal ---
        pred_lineal = linear_model.predict(datos_escalados)
        
        # --- Predicción con Modelo Polinómico ---
        # 2. Transformar los datos escalados a características polinómicas
        datos_polinomicos = poly_features.transform(datos_escalados)
        pred_polinomica = poly_model.predict(datos_polinomicos)
        
        # Devolver resultados en un formato claro
        resultados = {
            'prediccion_lineal': round(pred_lineal[0], 2),
            'prediccion_polinomica': round(pred_polinomica[0], 2)
        }
        
        return resultados

    except Exception as e:
        return f"Ocurrió un error durante la predicción: {e}"

In [5]:
print("\n\n--- PREDICCIÓN PARA EL ESTUDIANTE 1 ---")
resultado1 = predecir_calificacion_final(G1=16, G2=17, studytime=3, failures=0, absences=1)
if isinstance(resultado1, dict):
    print("\nResultados de la predicción:")
    print(f"  👨‍💻 Modelo Lineal predice una nota final de: {resultado1['prediccion_lineal']}")
    print(f"  🧠 Modelo Polinómico predice una nota final de: {resultado1['prediccion_polinomica']}")




--- PREDICCIÓN PARA EL ESTUDIANTE 1 ---

--------------------------------------------------
Datos de entrada del estudiante:
   G1  G2  studytime  failures  absences
0  16  17          3         0         1

Resultados de la predicción:
  👨‍💻 Modelo Lineal predice una nota final de: 17.45
  🧠 Modelo Polinómico predice una nota final de: 17.38


In [6]:
print("\n\n--- PREDICCIÓN PARA EL ESTUDIANTE 2 ---")
resultado2 = predecir_calificacion_final(G1=8, G2=9, studytime=2, failures=2, absences=10)
if isinstance(resultado2, dict):
    print("\nResultados de la predicción:")
    print(f"  👨‍💻 Modelo Lineal predice una nota final de: {resultado2['prediccion_lineal']}")
    print(f"  🧠 Modelo Polinómico predice una nota final de: {resultado2['prediccion_polinomica']}")



--- PREDICCIÓN PARA EL ESTUDIANTE 2 ---

--------------------------------------------------
Datos de entrada del estudiante:
   G1  G2  studytime  failures  absences
0   8   9          2         2        10

Resultados de la predicción:
  👨‍💻 Modelo Lineal predice una nota final de: 8.19
  🧠 Modelo Polinómico predice una nota final de: 8.65
